#### Import Statements

In [1]:
import pandas as pd
import scipy.sparse as sp
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource

#### Loading the data into dataframes

In [2]:
actors=pd.read_csv('actors.csv',index_col=0)
links=pd.read_csv('links.csv',index_col=0)
movies=pd.read_csv('movies.csv',index_col=0)

In [3]:
actors.head(5)

,actor_imdb_id,actor_name
0,287054,Julie Forsyth
1,2553224,Roy Entwistle
2,694286,Richard Powell
3,166612,Clark Clifford
4,288,Christian Bale


In [4]:
links.head(5)

,actor,movie
0,20277,778
1,13197,778
2,1705,778
3,8584,778
4,20194,778


In [5]:
movies.head(5)

,movie_imdb_id,movie_name,movie_rating,movie_vote,movie_year
0,3165,The Dead Man Who Killed,6.8,259,1913
1,3037,Juve Against Fantomas,6.8,376,1913
2,3930,Fantomas Against Fantomas,6.9,364,1914
3,2844,Fantomas - A l'ombre de la guillotine,6.9,486,1913
4,6206,Les vampires,7.0,1491,1915


##### (a) Let be the matrix denoted by if movie contains actor/C m × n Cij = 1 i actress j . Let B be the m × m matrix where Bij is the number of actors starring in both movie i and movie j .
(i) Show how to calculate B from C .
(ii) Show how to get the PageRank matrix P from B .

Creating a new C matrix that is a mXn represents ones wherever the actor index is the column index and movie index is the row index.

In [6]:
#forming C matrix
#making a dictionary to represent actors and movies and their row number and column number
#Created if we need to reference actors ID using their index and similarly for movies
actorid_columnNumber={k:v for k,v in zip(actors.index,actors['actor_imdb_id'])}
movieid_rowNumber={k:v for k,v in zip(movies.index,movies['movie_imdb_id'])}

#row,column,data values stored in three list for C matrix
row=[]
column=[]
data=[]

#creating the C matrix
for _,linksRow in links.iterrows():
    row.append(linksRow['movie'])
    column.append(linksRow['actor'])
    data.append(1)

C=sp.csc_matrix((data, (row,column)))
C

<928x32326 sparse matrix of type '<class 'numpy.int32'>'
	with 40958 stored elements in Compressed Sparse Column format>

Creating the B matrix by taking the dot product of the C and its transpose which will give the number of actors in that movie pair for example B[i,j] and B[j,i] will equal to the number of actors common in movie_i and movie_j.

The diagonals B[i,i] represent the number of actors in that movie.

In [7]:
B=C@C.T
B

<928x928 sparse matrix of type '<class 'numpy.int32'>'
	with 29006 stored elements in Compressed Sparse Column format>

In [8]:
B.todense()

matrix([[ 9,  5,  6, ...,  0,  0,  0],
        [ 5,  7,  6, ...,  0,  0,  0],
        [ 6,  6,  7, ...,  0,  0,  0],
        ..., 
        [ 0,  0,  0, ..., 10,  0,  4],
        [ 0,  0,  0, ...,  0, 39,  2],
        [ 0,  0,  0, ...,  4,  2, 18]], dtype=int32)

##### (b) Write the code to construct the PageRank matrix P using the language of your choice (Python, R, etc.).

Page Rank matrix(Google matrix) is the matrix that sums up each column and divide each element of that column 

In [9]:
#Page rank matrix
P=B/B.sum(axis=0)
P

matrix([[ 0.33333333,  0.2       ,  0.22222222, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.18518519,  0.28      ,  0.22222222, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.22222222,  0.24      ,  0.25925926, ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,  0.625     ,
          0.        ,  0.08510638],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.56521739,  0.04255319],
        [ 0.        ,  0.        ,  0.        , ...,  0.25      ,
          0.02898551,  0.38297872]])

##### (c) Compute the PageRank vector x of the movies via a Jacobi iteration that you write yourselfand normalize this quantity so ||x||1 = 1 . Decide on an intelligent measure for convergence (assume you do not know the actual PageRank vector x because your system is too large) Explain your choice of convergence criterion. Next, plot this convergence measure over the steps in your iteration. How many iterations does it take your implementation to get to a tolerance of 10−4 .

#### Algorithm

Now using the formula for linear system page rank
$$ (I-\alpha{P})x = v $$

P = Page rank matrix

Let  $ P_{hat} = (I-\alpha{P}) $

Let us assume that:
$$C=\begin{bmatrix} 0 & 1 & 0 \\ 1 & 0 & 1\\0 & 1 & 0\\ 1 & 1 & 1 \end{bmatrix}$$

then,
$$B=CC^T$$

$$B=\begin{bmatrix} 0 & 1 & 0 \\ 1 & 0 & 1\\0 & 1 & 0\\ 1 & 1 & 1 \end{bmatrix} . \begin{bmatrix} 0 & 1 & 0 & 1\\ 1 & 0 & 1 & 1\\0 & 1 & 0 & 1 \end{bmatrix}$$

$$B=\begin{bmatrix} 1 & 0 & 1 & 1\\ 0 & 2 & 0 & 2 \\1 & 0 & 1 & 1\\ 1 & 2 & 1 & 3 \end{bmatrix}$$

Now for getting the Page Rank Matrix we divide each column by its sum

Here we are assuming that all the movies represent a graph and the matrix B represent the number of edges between 

$$P^T=\begin{bmatrix} \dfrac{1}{3} & 0 & \dfrac{1}{3} & \dfrac{1}{3}\\ 0 & \dfrac{2}{4} & 0 & \dfrac{2}{4} \\\dfrac{1}{3} & 0 & \dfrac{1}{3} & \dfrac{1}{3}\\ \dfrac{1}{7} & \dfrac{2}{7} & \dfrac{1}{7} & \dfrac{3}{7} \end{bmatrix}$$

As we can see that this matrix is diagonally dominant and thus we can apply Jacobi iterative method, so we initialize the *x* as 1 divided by the number of movies.

$$x=\begin{bmatrix} \dfrac{1}{4} \\ \dfrac{1}{4}  \\\dfrac{1}{4} \\ \dfrac{1}{4}  \end{bmatrix}$$

Then we iterate using the formula $ (I-\alpha{P})x := x $

we also normalize this matrix using ||x||

To calculate tolerance I am calulcating the norm of the subtraction vetor of the new_x and the old_x $$ Tolerance =  ||x_{NEW}-x_{OLD} || $$


Covergence Criteria I am using currently is that when the tolerance goes below $10^{-6}$ we break the loop(of the jacobi iteration), **but after plotting the graph of all the iterations vs tolerance we can see that at about 200 iterations the tolerance almost becomes zero and saturate there we can use that as our convergence criteria.**

In [10]:
alpha=0.85

m=P.shape[0]

P_hat=np.eye(m)+alpha*P
x=np.ones((m,1))/m

flag=True
iteration_number=0
print1e4=True
toleranceAtEachIteration=[]


while(flag):
    
    old_x=x.copy()
    
    x=(P_hat)*x
    x=x/x.sum()
    
    new_x=x.copy()
    convergence_x=new_x-old_x
    tolerance=np.linalg.norm(convergence_x)
    
    iteration_number+=1
    
    if(tolerance<1e-4 and print1e4):
        iteration1e4=iteration_number
        print1e4=False
        
    if(tolerance<1e-6):
        flag=False
    toleranceAtEachIteration.append(tolerance)

print("The number of iterations to reach tolerance 10^-4: {} ".format(iteration1e4))
print("The number of iterations to converge: {} and Tolerance: {:f}".format(iteration_number,tolerance))

The number of iterations to reach tolerance 10^-4: 54 
The number of iterations to converge: 895 and Tolerance: 0.000001


Iteration vs Tolerance plot the following plot shows how the tolerance is dropping with iterations and looking at the graph we can see that tolerance saturates and almost becomes zero. We can choose the iteration number where the tolerance starts to saturate and we can choose that iteration number as our **convergence criteria**.

In [11]:
output_notebook()

hover = HoverTool(
        tooltips=[
            ("(Iteration,Tolerance)", "($x{int}, $y{1.111111})"),
        ]
    )

p = figure(title="Iteration Vs Tolerance",
     x_axis_label='Iteration',
     y_axis_label='Tolerance')

p.add_tools(hover)

p.line(x=list(range(len(toleranceAtEachIteration))),y=toleranceAtEachIteration)
show(p)

Loading BokehJS ...

In [17]:
#Flattening out the pageranks
A = np.squeeze(np.asarray(x))
A

array([  4.28676054e-04,   3.96459015e-04,   4.28257627e-04,
         4.12776037e-04,   5.57802368e-04,   5.39916767e-04,
         3.39419611e-04,   2.60276736e-04,   4.32037029e-04,
         6.55947153e-04,   2.21346285e-04,   3.69677456e-04,
         4.29140794e-04,   3.02038267e-04,   2.77151938e-04,
         7.17253961e-04,   4.15159007e-04,   2.65226310e-04,
         6.68039376e-04,   5.29545510e-04,   1.49905677e-04,
         4.83466497e-04,   4.03300716e-04,   2.88179125e-04,
         5.06589916e-04,   1.15132616e-04,   1.26621674e-04,
         4.14619224e-04,   5.06293683e-04,   6.44071659e-04,
         3.33606998e-04,   3.56341432e-04,   7.81361200e-04,
         5.05639906e-04,   5.51207501e-04,   5.05598207e-04,
         4.02145551e-04,   2.18280848e-04,   1.09117461e-03,
         5.39937276e-04,   1.01480084e-03,   3.44741693e-04,
         3.33629730e-04,   5.51435542e-04,   1.60806584e-04,
         5.74261046e-04,   1.07959029e-03,   4.13524686e-04,
         4.59333220e-04,

##### (d) List the PageRank values and titles of the 10 movies with the highest PageRank values.

#### Top 10 movies with highest page ranks

In [13]:
top10rankedmovies_indices=list(A.argsort()[-10:][::-1])
top10movies=movies.loc[top10rankedmovies_indices]
top10movies['Page Rank']=A[list(A.argsort()[-10:][::-1])]
top10movies.drop(['movie_vote','movie_year','movie_rating'],axis=1,inplace=True)
top10movies

,movie_imdb_id,movie_name,Page Rank
736,31679,Mr. Smith Goes to Washington,0.009829
751,27996,Mr. Deeds Goes to Town,0.008594
735,30993,You Can't Take It with You,0.007964
639,33467,Citizen Kane,0.007415
681,47522,A Star Is Born,0.006202
669,32551,The Grapes of Wrath,0.005756
666,35211,The Pride of the Yankees,0.005630
754,38650,It's a Wonderful Life,0.005573
689,35575,Yankee Doodle Dandy,0.005573
704,24216,King Kong,0.005275


##### (e) Plot IMDb movie rating vs. PageRank and IMDb movie votes vs. PageRank. Is PageRank a good predictor of movie rating or movie votes?

Adding page rank column to the movies dataframe (just for plotting purposes)

In [14]:
moviesWithPageRank=movies
moviesWithPageRank['Page Rank']=A

#### IMDB movie rating VS Page Rank

In [15]:
output_notebook()

source = ColumnDataSource(ColumnDataSource.from_df(moviesWithPageRank))

hover = HoverTool(
        tooltips=[
            ("Movie Name", "@movie_name"),
            ("(Rating,PageRank)", "($x{1.11}, $y{1.111111})"),
        ]
    )

p = figure(title="IMDb movie rating VS PageRank",
     x_axis_label='IMDb movie rating',
     y_axis_label='PageRank')

p.add_tools(hover)

p.scatter(x='movie_rating',y='Page Rank',source=source)
show(p)

Loading BokehJS ...

Looking at the graph we can see that the Page ranks are mostly normally distributed across the Movie Ratings and the most number of ratings are around 8.

#### IMDB movie votes VS Page Rank

In [16]:
output_notebook()

source = ColumnDataSource(ColumnDataSource.from_df(moviesWithPageRank))

hover = HoverTool(
        tooltips=[
            ("Movie Name", "@movie_name"),
            ("(votes,PageRank)", "($x{int}, $y{1.111111})"),
        ]
    )

p = figure(title="IMDb movie votes Vs PageRank",
     x_axis_label='IMDb movie votes',
     y_axis_label='PageRank')

p.add_tools(hover)

p.scatter(x='movie_vote',y='Page Rank',source=source)
show(p)

Loading BokehJS ...

Looking at the graph we can see that the number of movie votes does not have a very high correlation the page rank, this is expected because we have found the ranks based on the number of actors.